<a href="https://colab.research.google.com/github/bassiale/shape_recogniser/blob/main/shapes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/mydrive/MyDrive/Colab Notebooks/AI/datasets/data"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
)
'''train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    color_mode="rgb",
                                                    subset='training',
                                                    target_size=(128,128))'''

train_generator = tf.keras.preprocessing.image_dataset_from_directory(TRAINING_DIR,
                                                    batch_size=20,
                                                    
                                                    color_mode="rgb",
                                                    image_size=(128,128)
                                                    )

TEST_DIR = "/content/mydrive/MyDrive/Colab Notebooks/AI/datasets/datatest"

test_datagen = ImageDataGenerator(
    rescale=1./255,
    )
''''test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  batch_size=10,
                                                  class_mode='categorical',
                                                  color_mode="rgb",
                                                  classes=['cerchi','quadrati','triangoli'],
                                                  target_size=(128,128))'''
test_generator = tf.keras.preprocessing.image_dataset_from_directory(TEST_DIR,
                                                   batch_size=20,
                                                   
                                                   color_mode="rgb",
                                                   image_size=(128,128),
                                                    )
                        

Found 300 files belonging to 3 classes.
Found 30 files belonging to 3 classes.


In [ ]:
import urllib.request
import os
import zipfile
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape=(128, 128,3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False


last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output



last layer output shape:  (None, 6, 6, 768)


In [ ]:
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)

x = layers.Dense(3, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

history = model.fit(
            train_generator,
            validation_data=test_generator,
            epochs=20
            )

Epoch 1/20
15/15 [==============================] - 37s 2s/step - loss: 143.3299 - acc: 0.5433 - val_loss: 4.4328 - val_acc: 0.9333
Epoch 2/20
15/15 [==============================] - 14s 918ms/step - loss: 11.4871 - acc: 0.8433 - val_loss: 40.8274 - val_acc: 0.6667
Epoch 3/20
15/15 [==============================] - 14s 919ms/step - loss: 26.4283 - acc: 0.7367 - val_loss: 39.3173 - val_acc: 0.6667
Epoch 4/20
15/15 [==============================] - 14s 914ms/step - loss: 12.4922 - acc: 0.8467 - val_loss: 0.5214 - val_acc: 0.9667
Epoch 5/20
15/15 [==============================] - 14s 920ms/step - loss: 14.4192 - acc: 0.8933 - val_loss: 2.3604 - val_acc: 0.9333
Epoch 6/20
15/15 [==============================] - 14s 916ms/step - loss: 17.3113 - acc: 0.8300 - val_loss: 0.3279 - val_acc: 0.9667
Epoch 7/20
15/15 [==============================] - 14s 915ms/step - loss: 3.6673 - acc: 0.9200 - val_loss: 0.6460 - val_acc: 0.9667
Epoch 8/20
15/15 [==============================] - 14s 913ms/s

In [ ]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model/assets
